

# Acelerando Python

(Notebook tomada de Introducción a Python para ciencias e ingenierías (clase 6) - Ing. Martín Gaitán)

Algunas cosas que hay que saber:

- Python es un lenguaje de programación estandarizado que tiene **múltiples implementaciones**
- La implementación típica y oficial se llama estrictamente **CPython**, porque está programada en **C**
- Existen **otras implementaciones** del estándar: IronPython (.Net), Jython (Java) y hasta una implementación en una versión reducida Python llama **PyPy** (que, de paso, es rapidísima)
- Cada versión tiene sus propias ventajas, pero el hecho de que CPython sea C **permite extender con código muy eficiente hecho en C** (o C++)
- Pero **programar** en lenguajes de bajo nivel como **C es un *perno* **. 


## Repaso express de f2py


Ya vimos la herramienta F2py que aprovecha que Fortran y CPython se basan en C, y analizando el código fortran genera el "pegamento" (cabecesas y código C) para comunicar Python con Fortran. El resultado es un módulo (una extensión) importable desde python, donde las subrutinas fortran se convierten en funciones Python. 



In [ ]:
%%file suma.f90 

subroutine suma2(n, v)
    ! Compute the dot product between u and v (length n) and put the result in product
     
    integer, intent(in) :: n
    integer, intent(out) :: v
    v = n + 2
    
end subroutine

In [ ]:
# Si queres evitar el output podés guardar en una variable
_ = !f2py -m suma -c suma.f90   
#!/home/tin/.virtualenvs/curso/bin/f2py3.4 -m suma -c suma.f90 

In [ ]:
import suma
suma.suma2(10)

In [ ]:
%install_ext https://raw.github.com/mgaitan/fortran_magic/master/fortranmagic.py

In [ ]:
%load_ext fortranmagic

In [ ]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [ ]:
f1(4, 2)

## Cython

Cython es conceptualmente similar: **es un lenguaje** (con su correspondiente compilador), que extiende la sintaxis de Python (python con sabor a C). El **compilador Cython produce código C estándar** listo para ser compilado y usado como una extensión.

* código python válido es código cython válido
* Se pueden **llamar funciones en C**, o funciones/métodos de C++, **directamente** desde el código en Cython.   
* Es posible usar declarar tipos explícitamente (enteros, flotantes, o cualquier tipo de dato).

### Instalación


    conda install cython

o 

    pip install cython


### Hola mundo (versión larga)

Por convención, los archivos Cython se guardan con extensión .pyx  (viene de Pyrex, que es el paquete precedente)


In [ ]:
%%file helloworld.pyx

print('Hola Curso, soy Cython. #NoFueMagia')

Podemos compilar este código Cython para que genere el código C necesario para crear nuestra extensión Python

In [ ]:
!cython -a helloworld.pyx

In [ ]:
!cat helloworld.c

In [ ]:
from IPython.core.display import HTML
HTML(filename='helloworld.html')

Con el código C necesario, podemos compilar nuestra extensión

In [ ]:
!gcc -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing -I /usr/include/python3.5m -o helloworld.so helloworld.c

In [ ]:
import helloworld

## Hola mundo, the magic way

Pero si lo usamos en el notebook y lo modificamos seguido, es mejor usa el comándo magic **cython** 

In [ ]:
# %load_ext cythonmagic    en versiones anteriores a 0.21
%load_ext cython

In [ ]:
%%cython 

def hola():
    print('hola, soy una funcion cython')

print('Hola, soy cython, a través de un magic.')

In [ ]:
hola()

### Paréntesis: funciones generadoras

In [ ]:
def fib(n):
    """cálcula la serie hasta n y devuelve una lista"""
    serie = []
    a, b = 0, 1
    while b < n:
        serie.append(b)
        a, b = b, a + b
    return serie

fib(120)

In [ ]:
%timeit fib(1000000)

### El mismo generador en Cython

In [ ]:
%%cython

def fib_cython(n):
    """Print the Fibonacci series up to n."""
    a, b = 0, 1
    while b < n:
        yield b
        a, b = b, a + b

In [ ]:
%timeit list(fib_cython(100000))

### Cython on steroids

Pero veamos un código Cython que no sea Python puro. Supongamos que queremos calcular $\sum_{i=0}^n \alpha^i$ para $\alpha, n$ dados. 

Obviamente, antes de optimizar un algoritmo podriamos aplicar la formula de la progresión geométrica 

$$
\sum_{i=0}^n \alpha^i = \frac{1 - \alpha^{n+1}}{1 - \alpha}
$$

pero para "medir músculos" (?) lo haremos iterativamente

In [ ]:
def geo_prog(alpha, n):
    current = 1.0
    sum = current
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

In [ ]:
%timeit geo_prog(-4.0, 1000)

In [ ]:
%%cython 
def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current          # declaracion estática de tipo!
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum

In [ ]:
%timeit geo_prog_cython(-4.0, 1000)

# Numba

Numba es otra manera de optimizar código Python a través de un proceso de compilación. A diferencia de Cython, el proceso de compilación no produce código C válido para ser compilado como una extensión, sino que directamente se compila a "código maquina", pero *justo cuando se lo necesita* (ver [Just in Time compilation](http://en.wikipedia.org/wiki/Just-in-time_compilation))

Numba utiliza como tecnología subyacente la infraestructura de compilación [LLVM](http://en.wikipedia.org/wiki/LLVM) que es la encargada de realizar estas optimizaciones al vuelo 

### Instalación

Usando conda

    conda install numba
    
Via pip/virtualenv (linux)

    sudo apt-get install libllvm build-essential libllvm-dev libedit-dev
    pip install llvmlite
    pip install numba


In [ ]:
import numpy as np

In [ ]:
def sum2d(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [ ]:
x = np.ones((10, 20))

In [ ]:
sum2d(x)

In [ ]:
m_random = np.random.rand(1000, 1000)

In [ ]:
sum2d(m_random)

In [ ]:
%timeit sum2d(m_random)

Así se aplica la magia de Numba

In [ ]:
from numba import jit

sum2d_numba = jit(sum2d)

Para quienes desconfian, comprobemos que la funcion "numbizada" produce el mismo resultado

In [ ]:
sum2d_numba(m_random)

Pero anda ~10$^2$ más rápido

In [ ]:
%timeit sum2d_numba(m_random)

Una forma forma más elegante de usar usar `jit` es como un **decorador**

In [ ]:
@jit
def sum2d_(arr):
    M, N = arr.shape
    result = 0.0
    for i in range(M):
        for j in range(N):
            result += arr[i,j]
    return result

In [ ]:
%timeit sum2d_(m_random)

¡Más de 100 veces más rápido con una línea de código!

### Los límites de la magia de Numba y como maximizarla

Cuando no se definen tipos en la signatura, Numba hace una compilación *"lazy"* (vaga, que no se produce hasta el momento de se invoncada la función), e intentará operar sobre cualquier tipo de datos que reciba

In [ ]:
from numba import jit

@jit
def plus(x, y):
    # A somewhat trivial example
    return x + y

In [ ]:
plus(3.4, 3)

In [ ]:
plus('a', 'b')

Cuando especificamos la signatura, la optimización es *"eager"*, y obviamente esta especialización aumenta la performance sacrificando versatilidad

In [ ]:
@jit(int32(int32, int32))
def plus32(x, y):
    return x + y

Numba tiene **dos estrategias de compilación**: una optimización directa y de máxima perfomance denominado **modo nopython**, que genera código maquina que "no requiere consultar a la API C de Python" logrando inferir los tipos de datos con los que está trabajando. Cuando esta inferencia falla (cuando Numba no logró inferir cuál es el tipo de dato "dinámico" de alguna operación), automáticamente conmuta al **modo object** que sí utiliza la API C de Python. En este modo las únicas optimizaciones que se producen son con los *loops*. 



Pero la técnica de optimización es **factorizar el código "nopython"**. Para evitar que implicitamente se pase a modo object, se puede forzar el modo con `nopython=True`


In [ ]:
@jit(nopython=True)
def suma_nopy(x, y):
    return x + y